<a href="https://colab.research.google.com/github/wihika/INF0619/blob/main/INF0618_DeepLearning_Trabalho2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Trabalho 2 — INF-0618 Deep Learning   
Alunos:
- Bruno Kohn
- Lelia Lundi Andrade Loures
- Willian Hitoshi Kawakami


Neste trabalho, vamos utilizar o conjunto de dados Amazon Review, que consiste em resenhas de produtos classificadas por sentimentos (positivo ou negativo), sendo amplamente utilizado em tarefas de análise de sentimentos. O dataset contém 110.000 de resenhas, sendo 80.000 destinadas ao treinamento e 30.000 ao teste, com uma distribuição balanceada entre os sentimentos. Você é livre para utilizar o dataset completo ou trabalhar com amostras menores, como 5.000, 10.000 ou 25.000 resenhas, dependendo da capacidade computacional disponível. Se houver restrições computacionais, é recomendado o uso de um número menor de amostras. Caso contrário, sinta-se à vontade para explorar um volume maior de dados. Os arquivos de treinamento e teste serão disponibilizados no Moodle. A seguir, fornecemos um exemplo de como carregar esses dados.

## Instalação das Bibliotecas

Para instalar as bibliotecas necessárias, execute os comandos abaixo:

In [ ]:
!pip install pandas
!pip install transformers
!pip install bertviz

## Carregar e preparar os dados

Os arquivos CSV contêm três colunas: `polarity`, `title` e `text`. Essas colunas representam, respectivamente, o índice da classe (1 ou 2), o título da avaliação e o texto da avaliação.

- **polarity**: 1 para avaliações negativas e 2 para avaliações positivas
- **title**: título da avaliação
- **text**: conteúdo da avaliação

In [ ]:
import pandas as pd

# Carregando os dados de treinamento e teste
train_df = pd.read_csv("train.csv")  # Carregando dados de treino
test_df = pd.read_csv("test.csv")    # Carregando dados de teste

# Separação opcional por classes com base na coluna 'polarity'
# Filtrando o dataset de treino para a classe 'polarity' 1 e 2
train_class1 = train_df[train_df['polarity'] == 1]
train_class2 = train_df[train_df['polarity'] == 2]

# Amostragem aleatória opcional
# O aluno é livre para utilizar um número maior ou menor de amostras acima de 5000
train_class1_sample = train_class1.sample(n=5000, random_state=42)
train_class2_sample = train_class2.sample(n=5000, random_state=42)

## Exibir uma amostra por classe

Este código agrupa os dados pela coluna `polarity` e exibe uma amostra de cada classe.


In [ ]:
# Obter uma amostra por classe sem incluir a coluna de agrupamento 'polarity'
sample_per_class = train_df.groupby('polarity', group_keys=False).apply(lambda x: x.sample(1)).reset_index(drop=True)

# Mostrar as amostras
print(sample_per_class[['text', 'polarity']])

                                                text  polarity
0  I waited a long time for this box set to arriv...         1
1  this album has been named "album of the year" ...         2


<ipython-input-5-eefbb1dc0478>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_per_class = train_df.groupby('polarity', group_keys=False).apply(lambda x: x.sample(1)).reset_index(drop=True)


## Carregar modelo e tokenizer BERT para classificação

Neste exemplo, utilizamos a biblioteca `transformers` para carregar um modelo BERT pré-treinado e o tokenizer correspondente, configurado para uma tarefa de classificação binária. O número de classes é definido como 2 (positivo e negativo).

Você pode ajustar o modelo de acordo com sua capacidade computacional. Aqui estão as opções de modelos disponíveis:
- **"prajjwal1/bert-tiny"**: Modelo mais leve, ideal para recursos limitados.
- **"prajjwal1/bert-small"**: Um pouco maior, ainda eficiente para muitos cenários.
- **"prajjwal1/bert-base"**: Tamanho padrão do BERT.
- **"prajjwal1/bert-medium"**: Modelo mais pesado, adequado para máquinas mais potentes.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

NUM_LABELS = 2  # Número de classes
model_name = "prajjwal1/bert-tiny"  # Escolha o modelo conforme sua necessidade
tokenizer = AutoTokenizer.from_pretrained(model_name)  # Carregar o tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS)  # Carregar o modelo de classificação

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Atividade

Para o desenvolvimento deste trabalho, siga atentamente as atividades listadas a seguir. Em cada uma delas, espera-se que você apresente uma descrição detalhada dos experimentos realizados, acompanhada de uma discussão aprofundada dos resultados obtidos. Cada atividade possui uma pontuação específica, que será atribuída de acordo com a execução e a qualidade da análise apresentada.

Lembre-se de que a clareza na explicação dos métodos e a precisão na interpretação dos dados são essenciais para a avaliação. Bons resultados são importantes, mas a capacidade de refletir criticamente sobre eles também será considerada na pontuação.

Nota Importante: O conjunto de teste deve ser utilizado apenas uma vez, no final de todas as experimentações!

### Separação do Conjunto de Dados (Train.csv)
1.  (1.0) Utilizando o conjunto de dados train.csv, separe-o em duas partes: 80% para treinamento e 20% para validação.

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

 ### Tokenização das Resenhas de Filmes com Huggingface
 2. (1.0) Utilize a biblioteca Huggingface para carregar o tokenizador da variação do BERT que melhor se ajusta aos seus recursos computacionais e tokenizar as resenhas de filmes. As opções incluem bert-small, bert-tiny, bert-mini ou, para maior capacidade, google-bert/bert-base-uncased.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

NUM_LABELS = 2  #numero de classes
num_epochs = 3 #numero de éopocas
model_name = "prajjwal1/bert-tiny"  #modelo mais simples pois estamos rodando no Google
tokenizer = AutoTokenizer.from_pretrained(model_name)  #carrega o tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS)  #carregar o modelo de classificação

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Cálculo da Acurácia Balanceada no Conjunto de Validação (Baseline)
3.  (1.0) Baseado no modelo selecionado na atividade anterior, sem ajustes adicionais, calcule a acurácia balanceada no conjunto de validação. Esse valor servirá como ponto de referência (baseline) para futuras comparações.


In [ ]:
from torch import nn

def train(model, data_loader, optimizer):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs.logits, labels)
        loss.backward()
        optimizer.step()

In [ ]:
from sklearn.metrics import confusion_matrix

def balanced_accuracy(y_true, y_pred):
    #matriz de confusão
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    balanced_acc = (sensitivity + specificity) / 2

    return balanced_acc


In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(model, data_loader, labels_formated):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    print(f"Balanced Accuracy: {balanced_accuracy(labels_formated, predictions)}")
    return accuracy_score(actual_labels, predictions)

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf

#tokenize treino
train_texts = train_data['text'].tolist()
train_labels = train_data['polarity'].tolist()
train_labels_formated = [label - 1 for label in train_labels]
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors="pt", max_length=512)

#cria PyTorch DataLoader da treino
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels_formated))
train_loader = DataLoader(train_dataset, batch_size=32)

#tokenize validação
val_texts = val_data['text'].tolist()
val_labels = val_data['polarity'].tolist()
val_labels_formated = [label - 1 for label in val_labels]
val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors="pt", max_length=512)

#cria PyTorch DataLoader da validação
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(val_labels_formated))
val_loader = DataLoader(val_dataset, batch_size=32)  # Adjust batch size as needed

#treinamento
model.train()
for batch in train_loader:
    input_ids, attention_mask, labels = batch
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    loss = nn.CrossEntropyLoss()(outputs.logits, labels)
    loss.backward()

print(f"Treino")
accuracy = evaluate(model, train_loader, train_labels_formated)

#predição
model.eval()
predictions = []
with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        predictions.extend(predicted_labels.tolist())

print(f"Validação")
print(f"Balanced Accuracy: {balanced_accuracy(val_labels_formated, predictions)}")

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Treino
Balanced Accuracy: 0.5
Validação
Balanced Accuracy: 0.49993814943097475


Discussão:
Ajustar o parametro batch_size impactou na velocidade do treinamento, como isso impacta o uso de memoria usamos um valor baixo.

### Ajuste Fino (Fine-Tuning) do Modelo
4. (3.0) Efetue o ajuste fino (fine tuning) no modelo selecionado utilizando o conjunto de treinamento designado.
Experimente com duas taxas de aprendizado distintas: 0.01 e 0.00001 (embora outras taxas também possam ser exploradas).
Quais as acurácias balanceadas alcançadas no conjunto de validação para cada uma dessas taxas?
Analise e debata as variações de desempenho observadas entre elas.

In [ ]:
from transformers import TrainingArguments
from transformers import AdamW

learning_rates = [0.01, 0.00001]
models = [AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS),
          AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS)]

for index in range(len(learning_rates)):
    optimizer = AdamW(model.parameters(), lr=learning_rates[index])
    print(f"Learning Rate {learning_rates[index]}")
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(models[index], train_loader, optimizer)
        print(f"Treino")
        accuracy = evaluate(models[index], train_loader, train_labels_formated)
    models[index].eval()
    predictions = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            outputs = models[index](input_ids, attention_mask=attention_mask)
            predicted_labels = torch.argmax(outputs.logits, dim=1)
            predictions.extend(predicted_labels.tolist())
            print(f"Validação")
            print(f"Balanced Accuracy: {balanced_accuracy(val_labels_formated, predictions)}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Learning Rate 0.01
Epoch 1/3
Treino
Balanced Accuracy: 0.5000313322471488
Epoch 2/3
Treino
Balanced Accuracy: 0.5000313322471488
Epoch 3/3
Treino
Balanced Accuracy: 0.5000313322471488
Validação


ValueError: Found input variables with inconsistent numbers of samples: [16000, 32]

Discussão:

### Avaliação do Modelo com Melhor Desempenho no Conjunto Test.csv
5. (2.0) Utilize o modelo com melhor desempenho no conjunto test.csv e analise os resultados. Discuta brevemente os principais desafios enfrentados durante o treinamento do modelo e as soluções adotadas para superá-los. Além disso, reveja os hiperparâmetros escolhidos e reflita sobre o impacto potencial deles no desempenho do modelo.

In [ ]:
i = 1
best_learning_rate = learning_rates[i]
best_model = model[i]

test_data = pd.read_csv("test.csv")

#tokenize teste
test_texts = test_data['text'].tolist()
test_labels = test_data['polarity'].tolist()
test_labels_formated = [label - 1 for label in test_labels]
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt", max_length=512)

#cria PyTorch DataLoader da teste
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels_formated))
test_loader = DataLoader(test_dataset, batch_size=32)

#predição
best_model.eval()
predictions = []
with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        outputs = best_model(input_ids, attention_mask=attention_mask)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        predictions.extend(predicted_labels.tolist())

print(f"Balanced Accuracy: {balanced_accuracy(val_labels_formated, predictions)}")

Discussão:

### Análise do Mecanismo de Atenção com BertViz
6. (2.0) Use a ferramenta BertViz (github.com/jessevig/bertviz) para explorar os pesos do modelo selecionado na atividade anterior. Ao inspecionar o comportamento desse modelo, especificamente o transformador e seu mecanismo de atenção, quais padrões se tornam evidentes? Realize uma análise crítica desses padrões, destacando como eles podem impactar a interpretação e a performance do modelo.


In [ ]:
pip install bertviz

In [ ]:
from transformers import BertTokenizer, BertModel
from bertviz import head_view, model_view

#executar o melhor modelo
outputs = best_model(**test_encodings)
attentions = outputs.attentions

#visualiza as graficamente
head_view(attentions, test_encodings['input_ids'], tokenizer)

Discussão: